In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("/home/alex/dev/AiLearning/DataSetStore/text_classification_1/train.csv")
df_valid = pd.read_csv("/home/alex/dev/AiLearning/DataSetStore/text_classification_1/val.csv")

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181467 entries, 0 to 181466
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      181467 non-null  int64 
 1   text    181467 non-null  object
 2   class   181467 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ MB


In [4]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [5]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22683 entries, 0 to 22682
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      22683 non-null  int64 
 1   text    22683 non-null  object
 2   class   22683 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 531.8+ KB


In [6]:
df_valid.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [7]:
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')

# Just copy from the homework of the lesson -1
class MessageCleaner:
    
    def __init__(self,message=None):
        self._message= message
        self.tokens=[]

    def set_message(self,msg):
        self._message = msg;
        return self

    def get_message(self):
        return self._message

    def remove_users(self,replace=" "):
        xtr = re.sub(pattern="@[\w]*",repl=replace,string=self._message)
        self._message = xtr
        return self
    
    def remove_punctuation(self,replace=" "):
        xtr = re.sub(pattern=r'[^\w\s]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_en_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z0-9]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_ru_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^а-яА-Я0-9]',repl=replace,string=self._message)
        self._message = xtr
        return self
    
    def remove_en_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_ru_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^а-яА-Я]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def clean_by_regexp(self,regexp: str):
        pattern = re.compile(regexp)
        xtr = pattern.subn(' ', self._message)[0]
        self._message = xtr
        return self

    def to_lower (self ):
        self._message = self._message.lower();
        return self;

    def replace_by_dicts (self,dictionary: map):
        for key in dictionary:
            self._message = self._message.replace(key,dictionary[key])
            #re.sub(pattern=key,repl=dictionary[key],string=self._message)
        return self

    def escape_single_symbol_words (self):
        self._message=" ".join([word for word in self._message.split() if len(word)>1])
        return self

    def nltk_word_tokenize(self):
        self.tokens=tknz.word_tokenize(self._message)
        return self

    def nltk_word_punc_tokenize(self):
        self.tokens = tknz.wordpunct_tokenize(self._message)
        return self

    def nltk_tok_tok_tokenizer(self):
        self.tokens = tknz.ToktokTokenizer().tokenize(self._message)
        return self
        
    def nltk_tweet_tokenizer(self):
        self.tokens = tknz.TweetTokenizer().tokenize(self._message)
        return self

    def nltk_with_regexp_tokenizer (self,regexp):
        self.tokens = tknz.RegexpTokenizer(regexp).tokenize(self._message)
        return self

    def nltk_sentence_tokenizer (self):
        self.tokens = nltk.sent_tokenize(self._message)
        return self

    def tokenize (self,tokenizer):
        self.tokens=tokenizer(self._message)
        return self

    def remove_stopwords_from_tokens (self,lang=None,is_new=True):
        if lang is None:
            sw = set(stopwords.words("english"))
        else :
            sw = set(stopwords.words(lang))
        tks = [token for token in self.tokens if token not in sw]
        if is_new:
            self.tokens_without_stops = tks
        else :
            self.tokens = tks
        return self

    def stemme_tokens (self,stemmer=None,is_new=True):
        if stemmer is None:
            stemmer = PorterStemmer()
        xtr = [stemmer.stem(token) for token in self.tokens]
        if is_new :
            self.stem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def lematize_tokens (self,lematizer=None,is_new=True):
        if lematizer is None:
            lematizer = WordNetLemmatizer()
        xtr = [lematizer.lemmatize(token) for token in self.tokens]
        if is_new:
            self.lem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def process_each_token(self,function):
        xtr = [function(token) for token in self.tokens]
        self.tokens = xtr
        return self

    def replace_message_by_sorted_tokens(self,token_array=None):
        if token_array is None:
            token_array = self.tokens
        xtr = " ".join(sorted(token_array))
        self._message = xtr
        return self

[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
from natasha import Segmenter,MorphVocab,Doc,NewsMorphTagger,NewsEmbedding

segmenter = Segmenter()
morph_tagger = NewsMorphTagger(NewsEmbedding())
morph_vocab = MorphVocab()

def make_natasha_tokens(text,
                        morphology_filter_set=None,
                        token_length_limit=None,
                        empty_token="empty",
                        verbose=False):
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    selected_tokens=[]
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if verbose:
            print (" >> Input {} ".format(token))
        new_token = None
        if morphology_filter_set is None:
            new_token = token.lemma
        else:
            if token.pos in morphology_filter_set:
                new_token = token.lemma
        if new_token is not None and token_length_limit is not None:
            if len(new_token)<token_length_limit:
                new_token = None
        if new_token is not None:
            selected_tokens.append(new_token)
        if verbose:
            print (" >> Output {} ".format(new_token))
    if len(selected_tokens)==0 :
        selected_tokens.append(empty_token)

    del doc
    return selected_tokens

In [15]:

def natasha_message_cleaner (text: str):
    return MessageCleaner()\
    .set_message(msg=text)\
    .to_lower()\
    .tokenize(tokenizer=lambda txt: make_natasha_tokens(text=txt,\
                                                        morphology_filter_set=('NOUN','VERB','ADJ','PUNCT','PRON'),\
                                                        token_length_limit=None))\
    .tokens

def nltk_message_cleaner(text: str):
    return MessageCleaner()\
    .set_message(msg=text)\
    .nltk_word_tokenize()\
    .lematize_tokens(is_new=False)\
    .remove_stopwords_from_tokens(lang="russian",is_new=False)\
    .tokens
    

# .nltk_word_tokenize()\
# .remove_punctuation()\   
# .remove_ru_numbers()\
# .remove_ru_special_sym()\
# .remove_ru_special_sym()\
# .lematize_tokens(is_new=False)\
# .remove_stopwords_from_tokens(lang="russian",is_new=False)\


In [16]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer


#num_words 	the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
#filters 	a string where each element is a character that will be filtered from the texts. The default is all punctuation, plus tabs and line breaks, minus the ' character.
#lower 	    boolean. Whether to convert the texts to lowercase.
#split 	    str. Separator for word splitting.
#char_level if True, every character will be treated as a token.
#oov_token 	if given, it will be added to word_index and used to replace out-of-vocabulary words during text_to_sequence calls
#analyzer 	function. Custom analyzer to split the text. The default analyzer is text_to_word_sequence 

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=None,
                      filters=None,
                      lower=None,
                      split=None,
                      char_level=False,
                      oov_token=None,
                      analyzer=natasha_message_cleaner)

print(tokenizer.to_json())

tokenizer.fit_on_texts(["Ну что сказать, я вижу","Кто-то наступил на грабли", "Ты разочаровал меня", "ты был натравлен"])
print("Document numbre = ",tokenizer.document_count)
print("Word counter: ",tokenizer.word_counts)
print("Word into docs counter: ",tokenizer.word_docs)
print("Index docs: ",tokenizer.index_docs)
print("Index word: ",tokenizer.index_word)
print("Word index: ",tokenizer.word_index)

print(tokenizer.texts_to_sequences(["Ну что сказать, я вижу кто-то наступил"]))


{"class_name": "Tokenizer", "config": {"num_words": null, "filters": null, "lower": null, "split": null, "char_level": false, "oov_token": null, "document_count": 0, "word_counts": "{}", "word_docs": "{}", "index_docs": "{}", "index_word": "{}", "word_index": "{}"}}
Document numbre =  4
Word counter:  OrderedDict([('что', 1), ('сказать', 1), (',', 1), ('я', 2), ('видеть', 1), ('кто-то', 1), ('наступить', 1), ('грабли', 1), ('ты', 2), ('разочаровать', 1), ('натравить', 1)])
Word into docs counter:  defaultdict(<class 'int'>, {',': 1, 'я': 2, 'сказать': 1, 'видеть': 1, 'что': 1, 'наступить': 1, 'грабли': 1, 'кто-то': 1, 'разочаровать': 1, 'ты': 2, 'натравить': 1})
Index docs:  defaultdict(<class 'int'>, {5: 1, 1: 2, 4: 1, 6: 1, 3: 1, 8: 1, 9: 1, 7: 1, 10: 1, 2: 2, 11: 1})
Index word:  {1: 'я', 2: 'ты', 3: 'что', 4: 'сказать', 5: ',', 6: 'видеть', 7: 'кто-то', 8: 'наступить', 9: 'грабли', 10: 'разочаровать', 11: 'натравить'}
Word index:  {'я': 1, 'ты': 2, 'что': 3, 'сказать': 4, ',': 5, '

In [18]:
train_messages = df_train['text'].values
valid_messages = df_valid['text'].values

ANALYZER = natasha_message_cleaner

tokenizer = Tokenizer(num_words=None,
                      filters=None,
                      lower=None,
                      split=None,
                      char_level=False,
                      oov_token=None,
                      analyzer=ANALYZER)

tokenizer.fit_on_texts(train_messages)

train_vectors = tokenizer.texts_to_sequences(train_messages)
valid_vectors = tokenizer.texts_to_sequences(valid_messages)

max_vector_length = max([len(ANALYZER(train_message)) for train_message in train_messages])
dictionary_size_plus_1 = len(tokenizer.index_word) + 1


# https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
# just adding zeroes to vectors with small sizes
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(train_vectors, maxlen=max_vector_length)
x_valid = pad_sequences(valid_vectors, maxlen=max_vector_length)
y_train = df_train['class'].values
y_valid = df_valid['class'].values

print (max_vector_length,dictionary_size_plus_1)
print (x_train.shape)


130 93277
(181467, 130)


In [19]:
# https://keras.io/api/layers/core_layers/embedding/
from keras.layers import Embedding
from keras.models import Sequential

#    input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
#    output_dim: Integer. Dimension of the dense embedding.
#    embeddings_initializer: Initializer for the embeddings matrix (see keras.initializers).
#    embeddings_regularizer: Regularizer function applied to the embeddings matrix (see keras.regularizers).
#    embeddings_constraint: Constraint function applied to the embeddings matrix (see keras.constraints).
#    mask_zero: Boolean, whether or not the input value 0 is a special "padding" value that should be masked out. This is useful when using recurrent layers which may take variable length input. If this is True, then all subsequent layers in the model need to support masking or an exception will be raised. If mask_zero is set to True, as a consequence, index 0 cannot be used in the vocabulary (input_dim should equal size of vocabulary + 1).

embedding = Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True
)
model = Sequential()
model.add(embedding)

print (model.predict(x_train[0]).shape)
print (x_train[0].shape)



2024-01-03 21:59:56.826124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 21:59:57.915990: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


5/5 [==============================] - 0s 3ms/step
(130, 130)
(130,)


In [20]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import SimpleRNN,Dropout,Dense,BatchNormalization
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError

#https://keras.io/api/layers/recurrent_layers/simple_rnn/

rnn = Sequential()
rnn.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))

rnn.add(SimpleRNN(units=max_vector_length,dropout=0.25,recurrent_dropout=0.4))
rnn.add(Dense(96, activation='relu'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.75))
rnn.add(Dense(96, activation='tanh'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.5))
rnn.add(Dense(96, activation='relu'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.25))
rnn.add(Dense(96, activation='tanh'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.15))
rnn.add(Dense(1, activation='sigmoid'))

rnn.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['accuracy'])
rnn.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=20,
                             restore_best_weights=True,
                             start_from_epoch=3,
                             mode='max',
                             verbose=1)  

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 130)         12126010  
                                                                 
 simple_rnn (SimpleRNN)      (None, 130)               33930     
                                                                 
 dense (Dense)               (None, 96)                12576     
                                                                 
 batch_normalization (Batch  (None, 96)                384       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 96)                9312      
                                                      

In [21]:
history = rnn.fit(x_train, y_train,
                    batch_size=2000,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
73/73 [==============================] - 36s 460ms/step - loss: 0.2574 - accuracy: 0.5521 - val_loss: 0.0973 - val_accuracy: 0.9179
Epoch 2/50
73/73 [==============================] - 33s 459ms/step - loss: 0.0723 - accuracy: 0.9013 - val_loss: 0.0538 - val_accuracy: 0.9439
Epoch 3/50
73/73 [==============================] - 33s 458ms/step - loss: 0.0222 - accuracy: 0.9729 - val_loss: 0.0360 - val_accuracy: 0.9619
Epoch 4/50
73/73 [==============================] - 34s 459ms/step - loss: 0.0158 - accuracy: 0.9812 - val_loss: 0.0340 - val_accuracy: 0.9640
Epoch 5/50
73/73 [==============================] - 33s 459ms/step - loss: 0.0131 - accuracy: 0.9847 - val_loss: 0.0280 - val_accuracy: 0.9703
Epoch 6/50
73/73 [==============================] - 34s 459ms/step - loss: 0.0106 - accuracy: 0.9878 - val_loss: 0.0182 - val_accuracy: 0.9810
Epoch 7/50
73/73 [==============================] - 33s 459ms/step - loss: 0.0094 - accuracy: 0.9891 - val_loss: 0.0154 - val_accuracy: 0.9836

In [22]:
score = rnn.evaluate(x_valid, y_valid, batch_size=512, verbose=1)
print (score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 35ms/step - loss: 0.0151 - accuracy: 0.9840
[0.01509775873273611, 0.9839527606964111]
Test score: 0.01509775873273611
Test accuracy: 0.9839527606964111


In [23]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

# https://keras.io/api/layers/recurrent_layers/lstm/

lstm = Sequential()
lstm.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))

lstm.add(LSTM(units=max_vector_length,dropout=0.3,activation="tanh",
    recurrent_activation="tanh",recurrent_dropout=0.3))
lstm.add(Dense(96, activation='relu'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.65))
lstm.add(Dense(96, activation='tanh'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.45))
lstm.add(Dense(96, activation='relu'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.20))
lstm.add(Dense(96, activation='tanh'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.15))
lstm.add(Dense(1, activation='sigmoid'))

lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=20,
                             restore_best_weights=True,
                             start_from_epoch=3,
                             mode='max',
                             verbose=1)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 130)         12126010  
                                                                 
 lstm (LSTM)                 (None, 130)               135720    
                                                                 
 dense_5 (Dense)             (None, 96)                12576     
                                                                 
 batch_normalization_4 (Bat  (None, 96)                384       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 96)                0         
                                                                 
 dense_6 (Dense)             (None, 96)                9312      
                                                      

In [ ]:
history = lstm.fit(x_train, y_train,
                    batch_size=2000,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
73/73 [==============================] - 97s 1s/step - loss: 0.2176 - accuracy: 0.8960 - val_loss: 0.9404 - val_accuracy: 0.4930
Epoch 2/50
73/73 [==============================] - 93s 1s/step - loss: 0.0387 - accuracy: 0.9869 - val_loss: 1.1506 - val_accuracy: 0.4930
Epoch 3/50
73/73 [==============================] - 93s 1s/step - loss: 0.0205 - accuracy: 0.9938 - val_loss: 1.0157 - val_accuracy: 0.4935
Epoch 4/50
73/73 [==============================] - 93s 1s/step - loss: 0.0145 - accuracy: 0.9957 - val_loss: 0.7832 - val_accuracy: 0.5312
Epoch 5/50
73/73 [==============================] - 93s 1s/step - loss: 0.0138 - accuracy: 0.9958 - val_loss: 0.3158 - val_accuracy: 0.8207
Epoch 6/50
73/73 [==============================] - 93s 1s/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 0.0996 - val_accuracy: 0.9640
Epoch 7/50
73/73 [==============================] - 93s 1s/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0344 - val_accuracy: 0.9883
Epoch 8/50
73/73 [==